In [3]:
#Import Tools
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import openpyxl

In [39]:
#retrieve connections under radius=x
url = "https://raw.githubusercontent.com/erengozdeanil/UF-DecisionMaker/main/Edges.txt"
resp = requests.get(url)
edges = json.loads(resp.text)
#converts nested lists into a list of tuples
edges = [tuple(i) for i in edges]
print(tuple(edges))

(('V0', 'WO0'), ('V1', 'WO0'), ('V2', 'WO2'))


In [21]:
#retrieve data regarding vacant spaces
from openpyxl import load_workbook
workbook = load_workbook(filename="vacant spaces.xlsx")
workbook.sheetnames
sheet = workbook.active
sheet

#create dictionary based on excel worksheet
vacant_spaces={}
for value in sheet.iter_rows(min_row=2, values_only=True):
    for index,item in enumerate(value):
        vacant_spaces[value[0]]={}
        vacant_spaces[value[0]]["tag"]=value[1]
        vacant_spaces[value[0]]["building"]=value[2]
        vacant_spaces[value[0]]["size"]=value[3]
        vacant_spaces[value[0]]["loc_building"]=value[4]
        vacant_spaces[value[0]]["orientation"]=value[5]
        vacant_spaces[value[0]]["node_type"]=value[6]

#interpret data
#Rules:
#structure (roof:1,ground:3,outside:3,basement:3,intermediate:2)
#solar (roof:3,
#       basement:1
#       ground & S,SW,SE:3,
#       ground $ E,W:2
#       ground $ N,NE,NW:1
#       outside & S,SW,SE:3,
#       outside $ E,W:2
#       outside $ N,NE,NW:1
#       intermediate & S,SW,SE:3,
#       intermediate & E,W:2
#       intermediate & N,NE,NW:1

for space in vacant_spaces:
    if vacant_spaces[space]["loc_building"]=="roof":
        vacant_spaces[space]["structure"]=1
        vacant_spaces[space]["solar"]=3
    if vacant_spaces[space]["loc_building"]=="basement":
        vacant_spaces[space]["structure"]=3
        vacant_spaces[space]["solar"]=1
    if (vacant_spaces[space]["loc_building"]=="ground") and (((vacant_spaces[space]["orientation"])=="S") or ((vacant_spaces[space]["orientation"])=="SE") or ((vacant_spaces[space]["orientation"])=="SW")):
        vacant_spaces[space]["solar"]=3
    if (vacant_spaces[space]["loc_building"]=="ground") and (((vacant_spaces[space]["orientation"])=="N") or ((vacant_spaces[space]["orientation"])=="NE") or ((vacant_spaces[space]["orientation"])=="NW")):
        vacant_spaces[space]["solar"]=1
    if (vacant_spaces[space]["loc_building"]=="ground") and (((vacant_spaces[space]["orientation"])=="E") or ((vacant_spaces[space]["orientation"])=="W")):
        vacant_spaces[space]["solar"]=2
    if (vacant_spaces[space]["loc_building"]=="intermediate") and (((vacant_spaces[space]["orientation"])=="S") or ((vacant_spaces[space]["orientation"])=="SE") or ((vacant_spaces[space]["orientation"])=="SW")):
        vacant_spaces[space]["solar"]=3
    if (vacant_spaces[space]["loc_building"]=="intermediate") and (((vacant_spaces[space]["orientation"])=="N") or ((vacant_spaces[space]["orientation"])=="NE") or ((vacant_spaces[space]["orientation"])=="NW")):
        vacant_spaces[space]["solar"]=1
    if (vacant_spaces[space]["loc_building"]=="intermediate") and (((vacant_spaces[space]["orientation"])=="E") or ((vacant_spaces[space]["orientation"])=="W")):
        vacant_spaces[space]["solar"]=2
    if (vacant_spaces[space]["loc_building"]=="outside") and (((vacant_spaces[space]["orientation"])=="S") or ((vacant_spaces[space]["orientation"])=="SE") or ((vacant_spaces[space]["orientation"])=="SW")):
        vacant_spaces[space]["solar"]=3
    if (vacant_spaces[space]["loc_building"]=="outside") and (((vacant_spaces[space]["orientation"])=="N") or ((vacant_spaces[space]["orientation"])=="NE") or ((vacant_spaces[space]["orientation"])=="NW")):
        vacant_spaces[space]["solar"]=1
    if (vacant_spaces[space]["loc_building"]=="outside") and (((vacant_spaces[space]["orientation"])=="E") or ((vacant_spaces[space]["orientation"])=="W")):
        vacant_spaces[space]["solar"]=2
    

print(vacant_spaces)        

{'V1': {'tag': 'V1', 'building': 'BK', 'size': 3, 'loc_building': 'roof', 'orientation': 'SW', 'node_type': 'vacant', 'structure': 1, 'solar': 3}, 'V2': {'tag': 'V2', 'building': 'AA', 'size': 1, 'loc_building': 'outside', 'orientation': 'SE', 'node_type': 'vacant', 'solar': 3}, 'V3': {'tag': 'V3', 'building': 'BB', 'size': 2, 'loc_building': 'ground', 'orientation': 'S', 'node_type': 'vacant', 'solar': 3}, 'V4': {'tag': 'V4', 'building': 'CEG', 'size': 2, 'loc_building': 'basement', 'orientation': 'NE', 'node_type': 'vacant', 'structure': 3, 'solar': 1}, 'V5': {'tag': 'V5', 'building': 'TAG', 'size': 1, 'loc_building': 'intermediate', 'orientation': 'NW', 'node_type': 'vacant', 'solar': 1}, 'V6': {'tag': 'V6', 'building': 'BK', 'size': 3, 'loc_building': 'roof', 'orientation': 'SW', 'node_type': 'vacant', 'structure': 1, 'solar': 3}, 'V7': {'tag': 'V7', 'building': 'AA', 'size': 2, 'loc_building': 'outside', 'orientation': 'SE', 'node_type': 'vacant', 'solar': 3}, 'V8': {'tag': 'V8', 

In [ ]:
for value in sheet.iter_rows(min_row=3,
                             min_col=6,
                             values_only=True):
    if value[3] != None and value[1] != None and value[2] != None:
        new_value=[int(value[2]), value[1], value[0], int(int(value[3])/9), value[4]]
        if new_value[2]=="Shop":
            new_value[2]="1"
        elif new_value[2]=="Workshop":
            new_value[2]="3"
        elif new_value[2]=="Both shop and workshop":
            new_value[2]="2"
        if new_value[1]=="Daily":
            new_value[1]="D"
        elif new_value[1]=="Monthly":
            new_value[1]="M"
        elif new_value[1]=="Occationally":
            new_value[1]="O"
    
        p_list.append(tuple(new_value))